In [ ]:
import pandas as pd
corr_issues = pd.read_excel('meeting_corrosion_rows.xlsx')
corr_issues

In [ ]:
tower_corr = corr_issues['AMSAssetRef'].unique()
tower_corr

In [ ]:
df = pd.read_excel('Structure Details _ 17March2025.xlsx', sheet_name='Amplitel Structure Details')
df = df[df['AMSAssetRef'].isin(tower_corr)]

In [ ]:
df['InstallationDate']

In [ ]:
cutoff_date = pd.to_datetime('2001-12-31')

df = df[df['InstallationDate'] > cutoff_date]

In [ ]:
def convert_date(date_val):
    if pd.isna(date_val):
        return None  # or '' or np.nan, depending on your needs
    return date_val.strftime("%Y%m%d")

df['Formatted_Installation_Date'] = df['InstallationDate'].apply(convert_date)

In [ ]:
import urllib.request
import urllib.parse
import pandas as pd
from io import StringIO
import time

# Initialize a list to store individual DataFrames
daily_dfs = []

# API request setup
for i in df.index:
    api_url = 'https://www.longpaddock.qld.gov.au/cgi-bin/silo'
    
    params = {
        'format': 'standard',
        'lat': df['Latitude'][i],
        'lon': df['Longitude'][i],
        'start': df['Formatted_Installation_Date'][i],
        'finish': '20250620',
        'username': 'anubhav.jetley123@gmail.com',
        'password': 'apirequest'
    }
    url = api_url + '/DataDrillDataset.php?' + urllib.parse.urlencode(params)

    try:
        # Fetch data
        with urllib.request.urlopen(url) as remote:
            data = remote.read().decode('utf-8')  # Decode bytes to string

        # Find the start of the data table (skip metadata)
        lines = data.split('\n')
        data_start = 0
        for j, line in enumerate(lines):
            if line.startswith('Date       Day T.Max'):
                data_start = j
                break
        else:
            print(f"No data table found for index {i}, lat: {df['Latitude'][i]}, lon: {df['Longitude'][i]}")
            continue

        # Extract data lines (header + actual data)
        data_lines = lines[data_start:]

        # Join lines into a single string for Pandas
        data_str = '\n'.join(data_lines)

        # Read into DataFrame using space as delimiter
        single_df = pd.read_csv(StringIO(data_str), delim_whitespace=True)

        # Add a column to identify the source row (optional, for tracking)
        single_df['AMSAssetRef'] = df['AMSAssetRef'][i]
        single_df['Latitude'] = df['Latitude'][i]
        single_df['Longitude'] = df['Longitude'][i]

        # Append to the list
        daily_dfs.append(single_df)

        # Optional: Add a small delay to avoid overwhelming the API
        time.sleep(0.5)

    except Exception as e:
        print(f"Error processing index {i}, lat: {df['Latitude'][i]}, lon: {df['Longitude'][i]}: {e}")
        continue

# Concatenate all DataFrames
if daily_dfs:  # Check if the list is not empty
    final_df = pd.concat(daily_dfs, ignore_index=True)
    final_df.to_csv('silo.csv', index=False)
    print("Data saved to silo.csv")
else:
    print("No data was retrieved. Check API response or input data.")

In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import time
import pickle
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


# File to save progress
checkpoint_file = 'daily_dfs_checkpoint.pkl'
last_processed_file = 'last_processed_row.txt'

# Load existing progress if available
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'rb') as f:
        daily_dfs = pickle.load(f)
else:
    daily_dfs = []

# Load last processed row index, default to 0 if not exists
if os.path.exists(last_processed_file):
    with open(last_processed_file, 'r') as f:
        start_index = int(f.read())
else:
    start_index = 0

# Iterate through the DataFrame rows starting from last processed
for i in df.index[start_index:]:
    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": df['Latitude'][i],
            "longitude": df['Longitude'][i],
            "start_date": df['InstallationDate'],
            "end_date": "2025-06-20",
            "hourly": ["rain", "wind_speed_80m", "temperature_80m"]
        }
        responses = openmeteo.weather_api(url, params=params)

        # Process first location
        response = responses[0]

        # Process hourly data
        hourly = response.Hourly()
        hourly_rain = hourly.Variables(0).ValuesAsNumpy()
        hourly_wind_speed_80m = hourly.Variables(1).ValuesAsNumpy()
        hourly_temperature_80m = hourly.Variables(2).ValuesAsNumpy()

        # Create hourly DataFrame
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "rain": hourly_rain,
            "wind_speed_80m": hourly_wind_speed_80m,
            "temperature_80m": hourly_temperature_80m,
            "AMSAssetRef": df['AMSAssetRef'][i]
        }

        hourly_dataframe = pd.DataFrame(data=hourly_data)

        # Convert to daily averages
        daily_dataframe = hourly_dataframe.resample('D', on='date').agg({
            'rain': 'sum',  # Sum for rain as it’s cumulative
            'wind_speed_80m': 'mean',
            'temperature_80m': 'mean',
            'AMSAssetRef': 'first'
        }).reset_index()

        daily_dfs.append(daily_dataframe)

        # Save progress after each successful iteration
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(daily_dfs, f)
        
        # Update last processed row
        with open(last_processed_file, 'w') as f:
            f.write(str(i + 1))

    except Exception as e:
        print(f"Error at row {i}: {e}")
        # Wait for 60 seconds to respect API rate limit
        print("Waiting 60 seconds due to API limit...")
        time.sleep(60)  # Adjusted to 60 seconds as per your comment
        # Resume from the current row
        continue

# Concatenate all daily DataFrames
final_df = pd.concat(daily_dfs, ignore_index=True)
print(final_df)

# Optionally, save final DataFrame to CSV
final_df.to_csv('open_meteo_data.csv', index=False)